<a href="https://colab.research.google.com/github/drewbecker02/projects/blob/main/6560_HW_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Drew Becker

CSCI 6560

Homework 4

Nov 8

This is Notebook for 6560 Homework 4.
FOR DATA TABLES generated from the code, see https://docs.google.com/spreadsheets/d/1L_YqbMVQdz2Kb0EWMdrRs620l1la33pxFISkeAAMdI0/edit?usp=sharing.

To solve this homework, I used DEAP and implemented my own evolution strategies aglorithm (mu, lambda) with mu = 10 and lambda = 100, gens = 19. I also implemented an adaptive penalty that changes in response to feedback from the algorithm. Because this is an evolution strategy, mutation step sizes are coded into the genome, and natural selection adapts the mutation step sizes. This is a self-adaptive process. 

To Solve the ackley function, I used MU = 40 Lambda = 400, Gens = 50

Individual Representation: 2D array, with one dimension for doubles, representing objective values, and another dimension of doubles representing strategy values.

For all the following questions, I implemented the same crossover, selection, and mutation operators.


Operators: 

Parenthood selection: tournament size 10

Child selection: deterministic

Crossover: Blend crossover on both objective and stategy values

Mutation: Log normal muatation: the strategy 
variable is mutated by a log normal rule, and the objective value is mutated by a normal distriution.



In [ ]:
!pip install deap
!pip install matplotlib

import deap
from deap import creator
from deap import base
from deap import algorithms
from deap import tools
from deap import benchmarks
import math
from math import sin
import random
import numpy
from numpy import argsort
import array
import matplotlib.pyplot as plt

NGEN = 19
POP_SIZE = 10
CXPB = .9
MUTPB = .7
IND_SIZE = 2
K = 5
gens_passed = 0
penalty = 0
MIN_STRAT = .05
MAX_STRAT = 5
total_evals = 0

def varPenalty():
    beta = .2
    gamma = 1.5
    valid_count = 0
    invalid_count = 0
    penalty = .005 * gens_passed
    if gens_passed > 1:
        for champ in champs:
            if champ.fitness.valid:
                valid_count += 1
            else:
                invalid_count += 1
        if invalid_count == K:
            penalty = penalty * gamma
        elif valid_count == K:
            penalty = penalty * beta
    return penalty

def evaluate(ind):
    global total_evals
    score = 0
    x = ind[0]
    y = ind[1]
    score = (abs(x) + abs(y)) * ((1 + abs(sin(abs(x) * math.pi))) + abs(sin(abs(y) * math.pi)))
    if not feasible(ind):
        score += varPenalty()
    total_evals+=1
    return (score,)

def feasible(ind):
    x = ind[0]
    y = ind[1]
    if x < -60 or x > 40 or y < -30 or y > 70:
        return False
    return True


def initES(icls, scls, size, smin, smax):
    ind = icls(random.random() for _ in range(size))
    ind.strategy = scls(random.uniform(smin, smax) for _ in range(size))
    return ind

def checkStrategy(minstrategy):
    def decorator(func):
        def wrappper(*args, **kargs):
            children = func(*args, **kargs)
            for child in children:
                for i, s in enumerate(child.strategy):
                    if s < minstrategy:
                        child.strategy[i] = minstrategy
            return children
        return wrappper
    return decorator

def createEA():
    """Creates and runs the Evolutionary Algorithm. Much of the code was gathered from https://deap.readthedocs.io/"""
    global pop
    global hof
    global champs
    global gens_passed
    global total_evals
    champs = []
    #Making containers
    creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
    creator.create("Individual", array.array, typecode="d",
               fitness=creator.FitnessMin, strategy=None)
    creator.create("Strategy", array.array, typecode="d")

    #initialize and populate toolbox for individuals
    toolbox = base.Toolbox()
    toolbox.register("individual", initES, creator.Individual, creator.Strategy, IND_SIZE, MIN_STRAT, MAX_STRAT)
    #Set operators
    toolbox.register("mate", tools.cxESBlend, alpha =0.5)
    toolbox.register("mutate", tools.mutESLogNormal, c = 1, indpb = 0.4)
    toolbox.register("select", tools.selTournament, tournsize=10)
    toolbox.register("evaluate", evaluate)

    toolbox.decorate("mate", checkStrategy(MIN_STRAT))
    toolbox.decorate("mutate", checkStrategy(MIN_STRAT))
    #toolbox.decorate("evaluate", tools.DeltaPenalty(feasible, 5))
    
    
    #making population
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)
    pop = toolbox.population(n=POP_SIZE)
    hof = tools.HallOfFame(1)

    #Registering descriptive statistics
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("min", numpy.min)
    stats.register("avg", numpy.mean)
    stats.register("max", numpy.max)
    stats.register("std", numpy.std)

    MU, LAMBDA = 10, 100
    pop = toolbox.population(n=MU)
    hof = tools.HallOfFame(1)

    pop, logbook = algorithms.eaMuCommaLambda(pop, toolbox, mu=MU, lambda_=LAMBDA, 
        cxpb=0.6, mutpb=0.4, ngen=NGEN, stats=stats, halloffame=hof)


    # for g in range(NGEN):
    #     n_children = pop * 10
    #     next_gen = []
    #     # Select and clone the next generation individuals
    #     varPenalty()
    #     offspring = map(toolbox.clone, toolbox.select(pop, len(n_children)))

    #     # Apply crossover and mutation on the offspring
    #     offspring = algorithms.varAnd(offspring, toolbox, CXPB, MUTPB)

    #     # Evaluate the individuals with an invalid fitness
    #     invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
    #     fitnesses = toolbox.map(toolbox.evaluate, invalid_ind)
    #     for ind, fit in zip(invalid_ind, fitnesses):
    #         ind.fitness.values = fit

    #     offspring.sort(key=lambda x: x.fitness, reverse=True)
    #     next_gen = map(toolbox.clone, toolbox.select(offspring, len(pop)))

    #     # The population is entirely replaced by the offspring
    #     pop[:] = next_gen
    #     gens_passed +=1
    #     hof.update(pop)
    #     best_ind = pop[0]
    #     champs.append(best_ind)
    #     if len(champs) > K:
    #         champs.remove(champs[0])

    #Printing best individual and their fitness
    print ("Best Individual: ", hof[0])
    print ("Fitness: ", hof[0].fitness)
    print ("total evals: ", total_evals)
    total_evals = 0


def main():
    createEA()
main()


gen	nevals	min     	avg    	max    	std     
0  	10    	0.375431	2.05613	3.60855	0.959321
1  	100   	0.286286	0.799739	1.25607	0.336598
2  	100   	0.109741	0.26513 	0.463054	0.122772
3  	100   	0.0626957	0.0883899	0.139503	0.0275267
4  	100   	0.00758213	0.0434761	0.0626957	0.0150557
5  	100   	0.0174301 	0.0244765	0.0395362	0.0070287
6  	100   	0.00248206	0.00457596	0.0165256	0.0040456
7  	100   	0.000954537	0.00232051	0.00322421	0.000675378
8  	100   	0.000884859	0.00130871	0.00197457	0.000417824
9  	100   	0.000160412	0.000801921	0.00108248	0.000258699
10 	100   	0.000487393	0.000509823	0.000707182	6.58002e-05
11 	100   	0.000442377	0.000482892	0.000487393	1.35049e-05
12 	100   	0.00028235 	0.000388968	0.000487393	7.91455e-05
13 	100   	2.13081e-05	0.000232713	0.00028235 	7.68932e-05
14 	100   	2.13081e-05	0.000117079	0.00019969 	6.7439e-05 
15 	100   	1.75403e-05	4.21877e-05	0.00014612 	3.69523e-05
16 	100   	1.26751e-06	1.0966e-05 	2.01251e-05	7.13236e-06
17 	100   	1.19924e-06	2.

In [ ]:
!pip install deap
!pip install matplotlib

import deap
from deap import creator
from deap import base
from deap import algorithms
from deap import tools
from deap import benchmarks
import math
from math import sin
import random
import numpy
from numpy import argsort
import array
import matplotlib.pyplot as plt

NGEN = 19
POP_SIZE = 10
CXPB = .9
MUTPB = .7
IND_SIZE = 2
K = 5
gens_passed = 0
penalty = 0
MIN_STRAT = .05
MAX_STRAT = 5
total_evals = 0


def evaluate(ind):
    """Evaluation function for fitness"""
    global total_evals
    score = 0
    x = ind[0]
    y = ind[1]
    score = (abs(x) + abs(y)) * ((1 + abs(sin(abs(3*x) * math.pi))) + abs(sin(abs(3*y) * math.pi)))
    total_evals+=1
    return (score,)

def feasible(ind):
    """Checks if individual violates constraints"""
    x = ind[0]
    y = ind[1]
    if x < -60 or x > 40 or y < -30 or y > 70:
        return False
    return True


def initES(icls, scls, size, smin, smax):
    """Creates the individuals for the ES"""
    ind = icls(random.random() for _ in range(size))
    ind.strategy = scls(random.uniform(smin, smax) for _ in range(size))
    return ind

def checkStrategy(minstrategy):
    def decorator(func):
        def wrappper(*args, **kargs):
            children = func(*args, **kargs)
            for child in children:
                for i, s in enumerate(child.strategy):
                    if s < minstrategy:
                        child.strategy[i] = minstrategy
            return children
        return wrappper
    return decorator

def createEA():
    """Creates and runs the Evolutionary Algorithm. Much of the code was gathered from https://deap.readthedocs.io/"""
    global pop
    global hof
    global champs
    global gens_passed
    champs = []
    #Making containers
    creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
    creator.create("Individual", array.array, typecode="d",
               fitness=creator.FitnessMin, strategy=None)
    creator.create("Strategy", array.array, typecode="d")

    #initialize and populate toolbox for individuals
    toolbox = base.Toolbox()
    toolbox.register("individual", initES, creator.Individual, creator.Strategy, IND_SIZE, MIN_STRAT, MAX_STRAT)
    #Set operators
    toolbox.register("mate", tools.cxESBlend, alpha =0.5)
    toolbox.register("mutate", tools.mutESLogNormal, c = 1, indpb = 0.5)
    toolbox.register("select", tools.selTournament, tournsize=10)
    toolbox.register("evaluate", evaluate)

    toolbox.decorate("mate", checkStrategy(MIN_STRAT))
    toolbox.decorate("mutate", checkStrategy(MIN_STRAT))
    toolbox.decorate("evaluate", tools.DeltaPenalty(feasible, 5))
    
    
    #making population
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)
    pop = toolbox.population(n=POP_SIZE)
    hof = tools.HallOfFame(1)

    #Registering descriptive statistics
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("min", numpy.min)
    stats.register("avg", numpy.mean)
    stats.register("max", numpy.max)
    stats.register("std", numpy.std)

    MU, LAMBDA = 10, 100
    pop = toolbox.population(n=MU)
    hof = tools.HallOfFame(1)

    pop, logbook = algorithms.eaMuCommaLambda(pop, toolbox, mu=MU, lambda_=LAMBDA, 
        cxpb=0.6, mutpb=0.4, ngen=NGEN, stats=stats, halloffame=hof)


    #Printing best individual and their fitness
    print ("Best Individual: ", hof[0])
    print ("Fitness: ", hof[0].fitness)
    print ("total evals: ", total_evals)

def main():
  total_evals = 0
  for i in range(10):
      createEA()
main()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
gen	nevals	min    	avg    	max    	std     
0  	10    	1.67732	2.40006	3.65482	0.639072
1  	100   	0.227875	1.06994	1.49377	0.458219
2  	100   	0.0526056	0.261476	0.761532	0.218482
3  	100   	0.021185 	0.045388	0.0889675	0.0225816
4  	100   	0.00280281	0.0146675	0.0254734	0.00722032
5  	100   	0.00556265	0.011687 	0.0171056	0.00281716
6  	100   	0.00235177	0.00682624	0.0123955	0.00308281
7  	100   	0.00119726	0.00298399	0.00600992	0.00134924
8  	100   	0.000306361	0.00137018	0.00235177	0.000610886
9  	100   	9.91576e-06	0.000506186	0.000933647	0.00026081 
10 	100   	9.40167e-05	0.000267092	0.000452875	0.000103256
11 	100   	4.20942e-05	0.000129687	0.000227094	7.12745e-05
12 	100   	1.59826e-05	4.86519e-05	8.14982e-05	1.76731e-05
13 	100   	1.25984e-05	2.74652e-05	4.09572e-05	1.0458e-05 
14 	

Compared to the first function, It looks like the second function is much less narrow, considering answers that are very close together in Question 1 may have radically different fitness values. The increase in local maxima makes it harder to find a very good value, but overall i feel like the answers are more consistently good than in question 1.

In [ ]:
######################
##ACKLEY FUNCTION PROBLEM
#######################
!pip install deap
!pip install matplotlib

import deap
from deap import creator
from deap import base
from deap import algorithms
from deap import tools
from deap import benchmarks
import math
from math import sin
import random
import numpy
from numpy import exp
from numpy import sqrt
from numpy import cos
from numpy import e
from numpy import pi
from numpy import argsort
import array
import matplotlib.pyplot as plt

NGEN = 50
POP_SIZE = 40
CXPB = .9
MUTPB = .7
IND_SIZE = 30
K = 5
gens_passed = 0
penalty = 0
MIN_STRAT = .05
MAX_STRAT = 5
total_evals = 0


def ackley( x, a=20, b=0.2, c=2*pi ):
    x = numpy.asarray_chkfinite(x)  # ValueError if any NaN or Inf
    n = len(x)
    s1 = sum( x**2 )
    s2 = sum( cos( c * x ))
    return -a*exp( -b*sqrt( s1 / n )) - exp( s2 / n ) + a + exp(1)

def evaluate(ind):
    global total_evals
    score = 0
    x = ind[0]
    y = ind[1]
    score = ackley(ind)
    total_evals+=1
    return (score,)

def feasible(ind):
    for x in ind:
        if x < -30 or x > 30:
            return False
    return True

def initES(icls, scls, size, smin, smax):
    ind = icls(random.random() for _ in range(size))
    ind.strategy = scls(random.uniform(smin, smax) for _ in range(size))
    return ind

def checkStrategy(minstrategy):
    def decorator(func):
        def wrappper(*args, **kargs):
            children = func(*args, **kargs)
            for child in children:
                for i, s in enumerate(child.strategy):
                    if s < minstrategy:
                        child.strategy[i] = minstrategy
            return children
        return wrappper
    return decorator

def createEA():
    """Creates and runs the Evolutionary Algorithm. Much of the code was gathered from https://deap.readthedocs.io/"""
    global pop
    global hof
    global champs
    global gens_passed
    global total_evals
    champs = []
    #Making containers
    creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
    creator.create("Individual", array.array, typecode="d",
               fitness=creator.FitnessMin, strategy=None)
    creator.create("Strategy", array.array, typecode="d")

    #initialize and populate toolbox for individuals
    toolbox = base.Toolbox()
    toolbox.register("individual", initES, creator.Individual, creator.Strategy, IND_SIZE, MIN_STRAT, MAX_STRAT)
    #Set operators
    toolbox.register("mate", tools.cxESBlend, alpha =0.5)
    toolbox.register("mutate", tools.mutESLogNormal, c = 1, indpb = 0.4)
    toolbox.register("select", tools.selTournament, tournsize=10)
    toolbox.register("evaluate", evaluate)

    toolbox.decorate("mate", checkStrategy(MIN_STRAT))
    toolbox.decorate("mutate", checkStrategy(MIN_STRAT))
    toolbox.decorate("evaluate", tools.DeltaPenalty(feasible, 5))
    
    
    #making population
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)
    pop = toolbox.population(n=POP_SIZE)
    hof = tools.HallOfFame(1)

    #Registering descriptive statistics
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("min", numpy.min)
    stats.register("avg", numpy.mean)
    stats.register("max", numpy.max)
    stats.register("std", numpy.std)

    MU, LAMBDA = POP_SIZE, 10*POP_SIZE
    pop, logbook = algorithms.eaMuCommaLambda(pop, toolbox, mu=MU, lambda_=LAMBDA, 
        cxpb=0.7, mutpb=0.3, ngen=NGEN, stats=stats, halloffame=hof)

    #Printing best individual and their fitness
    print ("Best Individual: ", hof[0])
    print ("Fitness: ", hof[0].fitness)
    print ("total evals: ", total_evals)
    total_evals = 0


def main():
    for i in range(10):
        createEA()
main()


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
gen	nevals	min    	avg    	max    	std     
0  	40    	3.51588	3.90498	4.18764	0.161409
1  	400   	3.51924	3.76057	4.02751	0.132131
2  	400   	3.12707	3.59803	3.78677	0.119477
3  	400   	3.19195	3.42643	3.62427	0.0994472
4  	400   	3.01608	3.24002	3.43583	0.122474 
5  	400   	2.69905	3.01283	3.47981	0.159958 
6  	400   	2.51581	2.79953	3.05383	0.0949653
7  	400   	2.26013	2.5673 	2.87144	0.140622 
8  	400   	2.07276	2.33031	2.56334	0.138618 
9  	400   	1.76431	2.13757	2.39363	0.1349   
10 	400   	1.50949	1.87819	2.13341	0.171583 
11 	400   	1.23227	1.61207	1.85971	0.125503 
12 	400   	0.995363	1.36337	1.67122	0.148143 
13 	400   	0.966014	1.2138 	1.50206	0.0861868
14 	400   	0.866711	1.05119	1.25366	0.0942428
15 	400   	0.63857 	0.855103	1.01433	0.0827124
16 	400   	0.547213	0.72488 	0.89169